In [ ]:
import numpy as np
import control as con
from numpy import linalg as LA

import cvxpy
import optim_tools #own file with helper

import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
print cvxpy.installed_solvers()

In [ ]:
###########################
# Hydraulischer Aktor     #
###########################

A0 = np.matrix([[0,   1,       0],
                [-10, -1.167, 25],
                [0,   0,    -0.8]])
print "Eigenvalues: {}".format(LA.eigvals(A0))
#a = -A[-1,:].T ### !!!!!
#print a
b0 = np.matrix([[0],[0],[2.4]])
c0 = np.matrix([1, 0, 0])
d0 = np.matrix([0])
u_max = 10.5
n = 3

X00 = [np.matrix([-20.0, -10.0, -10.0]).T,
       np.matrix([-20.0, -10.0, 10.0]).T,
       np.matrix([-20.0,  10.0, -10.0]).T,
       np.matrix([20.0,  -10.0, 10.0]).T,
       np.matrix([20.0,  -10.0, -10.0]).T,
       np.matrix([20.0,   10.0, 10.0]).T]

#print "A:\n", A
#print "a:\n", a
#print "b:\n", b
#print "c:\n", c

# Convert to Normalform
(A1, b1, c1, d1), T1, Q1 = optim_tools.get_Steuerungsnormalform(A0, b0, c0.T, d0)
a1 = -A1[-1][:].T #!!!!
print "T1:\n", T1

# Convert to Normalform
ss, T2 = con.canonical_form(con.ss(A0, b0, c0, d0), form='reachable')

assert np.allclose(T1*X00[1],
                   optim_tools.reverse_x_order(T2*X00[1])),\
"own Steuerungsnormalform Transformation not equal python control version"
#print "x_r1:\n", T1*X00[1]
#print "x_r2(backwards):\n", optim_tools.reverse_x_order(T2*X00[1])

A = optim_tools.reverse_x_order(np.matrix(ss.A))
a = -A[-1][:].T #!!!!

b = optim_tools.reverse_x_order(np.matrix(ss.B))
c = optim_tools.reverse_x_order(np.matrix(ss.C))
d = optim_tools.reverse_x_order(np.matrix(ss.D)) # == 0!

print "A:\n", A
assert np.allclose(A, A1)
print "a:\n", a
assert np.allclose(a, a1)
print "b:\n", b
assert np.allclose(b, b1)
print "c:\n", c
assert np.allclose(c, c1.T)

X0 = [T1.dot(x0) for x0 in X00]
print "X0:\n", X0
#print "A1:\n", A1
#print "a1:\n", a1
#print "b1:\n", b1
#print "c1:\n", c1

In [ ]:
def someparameters():
    k0 = np.matrix([[ 0.00702601],
     [-0.79417547],
     [-0.01583519]])

    k1 = np.matrix([[ 0.00044966],
     [-0.02280205],
     [-0.00489574]])

    k0_star = np.matrix([[ 0.00151984],
     [-0.2060622 ],
     [-0.00826113]])

    k1_star = np.matrix([[ 0.00045032],
     [-0.02262498],
     [-0.00470835]])


In [ ]:
# Solution directly from polyplacement char. polynomial to wanted polynomial (fastest) with given polynomical coeff a
def k_explizit(roots_p1, roots_pmin, pmin, a):
    n = len(roots_p1)
    
    k0 = np.zeros((n,))
    k1 = np.zeros((n,))

    a_tilde_p1 = np.matrix(np.poly(roots_p1)[:0:-1]).T # get the characteristical polynomial backwards
    a_tilde_pmin = np.matrix(np.poly(roots_pmin)[:0:-1]).T # get the characteristical polynomial backwards
    
    k1 = (a_tilde_p1 - a_tilde_pmin) / (1.0-1.0/pmin)
    k0 = a_tilde_p1 - a - k1

    return np.matrix(k0).T, np.matrix(k1).T

%timeit k_explizit([1,2,4], [2,4,8], 0.1, a)

In [ ]:
# Do canonical form first (costy)
def k_explizit_x(roots_p1, roots_pmin, pmin, A, b):
    (A_R, _, _, _), _, _ = optim_tools.get_Steuerungsnormalform(A, b, b, 0)
    a = -A_R[-1][:].T
    n = len(roots_p1)
    
    k0 = np.zeros((n,))
    k1 = np.zeros((n,))

    a_tilde_p1 = np.matrix(np.poly(roots_p1)[:0:-1]).T # get the characteristical polynomial backwards
    a_tilde_pmin = np.matrix(np.poly(roots_pmin)[:0:-1]).T # get the characteristical polynomial backwards
    
    k1 = (a_tilde_p1 - a_tilde_pmin) / (1.0-1.0/pmin)
    k0 = a_tilde_p1 - a - k1

    return np.matrix(k0).T, np.matrix(k1).T

%timeit k_explizit_x([1,2,4], [2,4,8], 0.1, A, b)

In [ ]:
# Use python control to place poles and then interpolate (faster with A and b)
def k_explizit2(roots_p1, roots_pmin, pmin, A, b):
    r0 = con.place(A, b, roots_p1) #k(p=1)
    r1 = con.place(A, b, roots_pmin) #k(p=pmin)

    # This seems to work as expected
    k1 = 1.0/(1.0-1.0/pmin) * (r0 - r1)
    k0 = r0 - k1
    return np.matrix(k0), np.matrix(k1)

%timeit k_explizit2([1,2,4], [2,4,8], 0.1, A, b)

In [ ]:
### Design Sättigungsregler mittels konvexer Hülle (A.3)

# Variables (Convex)
#  Name in A.3 | Name in Program
#    Q  = P^⁻1 |   = Q
#    z  = Ql   |   = z0
#    z* = Ql*  |   = z1

# Variables (Quasi convex)
#    gamma     |   = g

# Parameter
#    mu        |   = m (Designparameter)
#    X0        |   = X0 = [x0,...]
#    A         |   = A
#    b         |   = b

# Initialize
n = len(b) # get dim of system

# Define Variables
#Q  = cvxpy.Semidef(n) #symmetric and positive semidefinite
Q  = cvxpy.Variable(n, n) # Semidef could go as an additional constraint as well, thereby no need fo Q to be symmetric

z0 = cvxpy.Variable(n)
z1 = cvxpy.Variable(n)

# Bisection parameter
g = cvxpy.Parameter(sign='positive')
#g.value = g_val # TODO set by bisection loop (function?)

m = cvxpy.Parameter(sign='positive')
#m.value = 1 # TODO: mu* >=1

# Define Constraints
const_sdQ = Q >> 0 # semidef but not (necessarily) symmetric

# (A.10)
const_A10 = [cvxpy.bmat([[Q,       X0[i]],
                         [X0[i].T, 1    ]]) >> 0
                            for i in range(0, len(X0))]

# (A.11)
const_A11 = cvxpy.bmat([[Q,  z0],
                        [z0.T, 1 ]]) >> 0

# (A.12)
const_A12 = cvxpy.bmat([[Q,    z1  ],
                        [z1.T,   m**2]]) >> 0

# (A.13)
const_A13 = Q*A + A.T*Q - b*z0.T - z0*b.T << 0 # This constraint is strict definit

# (A.14)
const_A14 = Q*A + A.T*Q - b*z1.T - z1*b.T << -2*g*Q # This constraint is strict definit
#const_A14 = Q*A + A.T*Q - b*z1.T - z1*b.T << cvxpy.bmat([[-2*g, 0, 0],
#                                                         [0, -2*g, 0],
#                                                         [0, 0, -2*g]])*Q # This constraint is strict definit


# Collect all constraints
constraints = [const_sdQ]
constraints.extend(const_A10) ##!! Beware of the "extend" if input is array
constraints.append(const_A12)
constraints.append(const_A13)
constraints.append(const_A14)


# Feasibility for bisection:
obj = cvxpy.Minimize(0)

# Stronger requirements for bisection? -> probably resulting in higher iterations, thus better results: TODO: Fix bisect!
obj_alt = cvxpy.Maximize(cvxpy.log_det(Q)) # Identical to geo_mean (in term of convexity and result)

# Form and solve problem.
prob = cvxpy.Problem(obj, constraints)

In [ ]:
"""
 Aemo of plotting complex functions in python.

 Jim M | Feb 2011 | GPL
"""
# Plotting functions ; see the example below
# and http://matplotlib.sourceforge.net/
#from matplotlib.pyplot import plot, legend

# Complex math (cmath) python functions ;
# see  see http://docs.python.org/library/cmath.html
#from cmath import sin, cos, exp, pi, log, polar, rect, phase, sqrt

# Note that python represents imaginary numbers like "3i" as "3j",
# where "j" meaning "sqrt(-1) must be preceded by a number,
# so "sqrt(-1)" alone would in python be "1j".
#
#     (3,4) complex rectangular form:     z = 3 + 4j
#     (x,y) complex rectangular form :    z = x + y * 1j
#     polar form :                        z = r * exp(1j * theta)
#     abs(z)   is length of complex number = r
#     phase(z) is angle of complex number = theta
#     z.real   is real part
#     z.imag   is imaginary part
#
# abs() is a python built-in; as are complex numbers themselves.
# But the other functions needed to be imported in their complex versions.
# The numeric constant pi can be imported from math or cmath.

# Remember that
# 1. lambda(x: ...) is an anyonymous function of x, e.g. lambda(x: 2*x+1)
# 2. map(f, [a, b, c, ...])  # returns [f(a), f(b), f(c), ...]


# == So here are a few utility functions for multiplying scalars and vectors.

# a scalar times a vector returns a vector
def scale_vector(scale, vector):
  result = [0]*len(vector)
  for i in range(len(result)):
    result[i] = scale * vector[i]
  return result

# dot product of two vectors = sum(x[0]*y[0] + ... + x[n-1]*y[n-1])
def vector_dot(vector1, vector2):
  result = 0
  for i in range(len(vector1)):
    result += vector1[i] * vector2[i]
  return result

# return real part of a vector
def real_vector(vector):
  return map(lambda x: x.real, vector)

# return imaginary part of a vector
def imag_vector(vector):
  return map(lambda x: x.imag, vector)

In [ ]:
# TODO: zeta -> zeta0 AND zeta_* -> zeta1  ---> Should come from optimisation loop
zeta0 = 1.1  #More or less chosen randomly here
zeta1 = 2.0  #More or less chosen randomly here

m1 = 2 # mu_*
pmin = 0.01

In [ ]:
%%time
# Perform bisection on g for mu=1
m.value = 1
[[o_Q, o_z0, o_z1], o_g] = optim_tools.bisect_max(0, None, prob, g, [Q, z0, z1], bisect_verbose=True,
                                              solver=cvxpy.SCS, warm_start=True, max_iters=800000, verbose=False)

print "-----------RESULTS-----------"
# l*(mu=1 -> m0) -> l1m0
l1m0 = LA.inv(o_Q) * o_z1
print "l*(mu=1)=\n", l1m0

roots_m0_p1 = LA.eigvals(A-b*l1m0.T)
print "lambda^{hat}(p=1)    = ", roots_m0_p1

roots_m0_pmin = zeta0 * roots_m0_p1.real + 1j*roots_m0_p1.imag
#roots_m0_pmin = zeta0 * roots_m0_p1  # Shifting on both real and imaginary axis in zeta

print "lambda^{hat}(p=pmin) = ", roots_m0_pmin

In [ ]:
%%time
# Perform bisection on g for mu>=1
m.value = m1 # TODO: mu* comes from optimisation loop

[[o_Q, o_z0, o_z1], o_g] = optim_tools.bisect_max(0, None, prob, g, [Q, z0, z1], bisect_verbose=True,
                                              solver=cvxpy.SCS, warm_start=True, max_iters=1600000, verbose=False)

print "-----------RESULTS-----------"
# l*(mu=1 -> m0) -> l1m0
l1m1 = LA.inv(o_Q) * o_z1
print "l*(mu=mu_*={})=\n".format(m.value), l1m1

roots_m1_p1 = LA.eigvals(A-b*l1m1.T)
print "lambda^{hat}_{*}(p=1)    = ", roots_m1_p1

roots_m1_pmin = zeta1 * roots_m1_p1.real + 1j*roots_m1_p1.imag
#roots_m0_pmin = zeta1 * roots_m1_p1  # Shifting on both real and imaginary axis in zeta

print "lambda^{hat}_{*}(p=pmin) = ", roots_m1_pmin

In [ ]:
# Get Polynomcoefficiants
# k0_i -> k_i

k0_0, k0_1 = k_explizit2(roots_m0_p1, roots_m0_pmin, pmin, A, b)

plt.axis([-2, .1, -3.5, 3.5])
poles = []
for p in np.arange(1, pmin, -0.01):
    #sys_cl = control.ss(A-b*(r0), b, c.T, d)
    #poles2, zeros2 = control.matlab.pzmap(sys_cl, True)
    #sys_cl = control.ss(A-b*(r1), b, c.T, d)
    #poles2, zeros2 = control.matlab.pzmap(sys_cl, True)
    #print p
    sys_cl = con.ss(A-b*(k0_0+1.0/p*k0_1), b, c, d)
    pole, zeros = con.matlab.pzmap(sys_cl, True)
    #print pole[0]
    poles.append(pole)
    #print poles2
plt.show

In [ ]:
# Get Polynomcoefficiants
# k1_i -> k_{*,i}

k1_0, k1_1 = k_explizit2(roots_m1_p1, roots_m1_pmin, pmin, A, b)

plt.axis([-2, .1, -3.5, 3.5])
poles = []
for p in np.arange(1, pmin, -0.01):
    #sys_cl = control.ss(A-b*(r0), b, c.T, d)
    #poles2, zeros2 = control.matlab.pzmap(sys_cl, True)
    #sys_cl = control.ss(A-b*(r1), b, c.T, d)
    #poles2, zeros2 = control.matlab.pzmap(sys_cl, True)
    #print p
    sys_cl = con.ss(A-b*(k1_0+1.0/p*k1_1), b, c, d)
    pole, zeros = con.matlab.pzmap(sys_cl, True)
    #print pole[0]
    poles.append(pole)
    #print poles2
plt.show

In [ ]:
# A.4
# Transformationen


In [ ]:
import cvxpy

R0 = cvxpy.Variable(n,n)
R1 = cvxpy.Variable(n,n)

G = cvxpy.Variable(n,n) #skew
G_star = cvxpy.Variable(n, n) # skew

# D = cvxpy.Symmetric(n)
# D_star = cvxpy.Symmetric(n)
D = cvxpy.Variable(n, n) #sym
D_star = cvxpy.Variable(n, n) #sym

constraints = [D                 == D.T, # sym
               D_star            == D_star.T, # sym
               G + G.T           == 0, # skew
               G_star + G_star.T == 0] # skew


# Parameter
alpha = 1-p_min
beta = 1+p_min
    
A0 = A-b*k0.T
A1 = A-b*k1.T

S0 = A1.T*R1+R1*A1
S1 = A0.T*R1+R1*A0 + A1.T*R0+R0*A1
S2 = A0.T*R0+R0*A0

A0_star = A-b*k0_star.T
A1_star = A-b*k1_star.T

S0 = A1.T*R1+R1*A1
S1 = A0.T*R1+R1*A0 + A1.T*R0+R0*A1
S2 = A0.T*R0+R0*A0

S0_star = A1_star.T*R1+R1*A1_star
S1_star = A0_star.T*R1+R1*A0_star + A1_star.T*R0+R0*A1_star
S2_star = A0_star.T*R0+R0*A0_star

# Create two constraints.
const4_39ab = [R0 >> 0,
               R0 + R1 >> 0]

const4_39c = cvxpy.bmat([[-D , G],
                         [G.T, D]]) - \
             cvxpy.bmat([[S0 + 0.5*beta*S1 + 0.25*beta**2*S2, 0.25*alpha*(S1 + beta*S2)],
                         [0.25*alpha*(S1 + beta*S2),          0.25*alpha**2*S2]]) >> 0

const4_39d = [cvxpy.bmat([[1, X0[i].T],
                          [X0[i], R0+R1]]) >> 0
                              for i in range(0, len(X0))]

const4_39e = cvxpy.bmat([[1,    k0.T],
                         [k0,   R0]]) + \
             1/p_min * cvxpy.bmat([[0,  k1.T],
                                  [k1, R1]]) >> 0

const4_39f = cvxpy.bmat([[1,    k0.T],
                         [k0,   R0]]) + \
             cvxpy.bmat([[0,  k1.T],
                         [k1, R1]]) >> 0

const4_39g = cvxpy.bmat([[-D_star , G_star],
                         [G_star.T, D_star]]) - \
             cvxpy.bmat([[S0_star + 0.5*beta*S1_star + 0.25*beta**2*S2_star, 0.25*alpha*(S1_star + beta*S2_star)],
                         [0.25*alpha*(S1_star + beta*S2_star),          0.25*alpha**2*S2_star]]) >> 0

constraints.extend(const4_39ab) ##!! Beware of the "extend" if input is array
constraints.append(const4_39c)
constraints.extend(const4_39d) ##!! Beware of the "extend" if input is array
constraints.append(const4_39e)
constraints.append(const4_39f)
constraints.append(const4_39g)

# Form objective.
obj = cvxpy.Minimize(cvxpy.trace(R0+1/p_min*R1))

# Form and solve problem.
prob = cvxpy.Problem(obj, constraints)
prob.solve(solver=cvxpy.SCS, verbose=True, max_iters=2500)  # Returns the optimal value.
print "status:", prob.status
print "optimal value", prob.value
print "R0", R0.value
LA.cholesky(R0.value)
print "R1", R1.value
LA.cholesky(R1.value + R0.value)
print "ok"